# Tello Tempo

## Import de bibliothèques

In [2]:
!pip install tellopy
!pip install djitellopy

In [1]:
from djitellopy import Tello
import tellopy
import cv2 as cv 
import socket, time
from time import sleep
import datetime
import os
from threading import Thread
import numpy as np

import pytz # timezone for timestamp
import datetime


In [2]:
# Tello??

### Useful custom functions

In [3]:
def get_timestamp():
    tz = pytz.timezone('Europe/Paris')
    currentDT = datetime.datetime.now(tz)
    TIMESTAMP = currentDT.strftime("%Y%m%d_%H%M%S")
    return TIMESTAMP

In [4]:
def battery_level(tello):
    print(f"Battery level is {tello.get_battery()}%")

In [9]:
tello = Tello()
tello.connect(False)

# afficher niveau de batterie
battery_level(tello)


[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


Battery level is 47%


In [9]:
def difference(list1, list2):
    zip_object = zip(list1, list2)
    difference=[]
    for list1_i, list2_i in zip_object:
        difference.append(list2_i-list1_i)
    return difference

# Utilisation  de `Tellopy`

* Décollage et attérissage 

In [10]:

# Paramètres de communication
host = ''
port = 9000
locaddr = (host,port)

#Connexion à l'adresse IP du drône pour entamer la communication 
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
tello_address = ('192.168.10.1', 8889)
sock.bind(locaddr)


sock.sendto('command'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# # Takeoff
# sock.sendto('takeoff'.encode(encoding="utf-8"), tello_address)
time.sleep(5)


#Rotate clockwise 360
# sock.sendto('tcw 360'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Land
# sock.sendto('land'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Utilisation de  `Djitellopy`

##  Décollage , photo et attérissage 

In [16]:
tello= Tello () 
tello.connect() # Connexion

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


In [27]:
battery_level(tello)

Battery level is 91%


In [12]:
tello.takeoff() # Décollage 
# tello.streamon() # Activation de la caméra

time.sleep(1) # Tempo de 2s 
frame_read = tello.get_frame_read() # Capture d'impage
img_name = f"view_{get_timestamp()}.png"
print(img_name)
cv.imwrite("testnew.png", frame_read.frame) # Stockage en local 
time.sleep(1)
tello.streamoff()
tello.land()   # Attérissage

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'takeoff'
[INFO] tello.py - 461 - Response takeoff: 'ok'


KeyboardInterrupt: 

In [15]:
# battery_level(tello)

KeyError: '192.168.10.1'

*  Capture de vidéo

In [ ]:
tello = Tello()
tello.connect()

KeepRecording= True
tello.streamon()

frame_read = tello.get_frame_read()

def videoRecorder():
    height, width,_= frame_read.frame.shape
    video_name = f"video_{get_timestamp()}.avi"

    video = cv.VideoWriter(video_name, cv.VideoWriter_fourcc(*'XVID'), 30, (width,height))
    
    while KeepRecording:
        video.write(frame_read.frame)
        time.sleep(1/30)
    video.release()
    
recorder  = Thread(target=videoRecorder)
recorder.start()

tello.takeoff()
tello.land()
KeepRecording= False
recorder.join()



## Patrol - Tello.curve

In [31]:
def patrol():
    """
    Dessine un arc de cercle et se retourne
    Espae nécessaire environ 1.5m
    """
    tello.curve_xyz_speed(25, -25, 0, 150, 25, 0, 50)
    tello.rotate_clockwise(180)


In [1]:
tello = Tello()
tello.connect(False)

NameError: name 'Tello' is not defined

In [35]:
tello.land()

[INFO] tello.py - 437 - Send command: 'land'
[INFO] tello.py - 461 - Response land: 'ok'


In [33]:
NUM_PATROL = 4

# afficher niveau de batterie
battery_level(tello)

tello.takeoff()
tello.move_up(80)

for k in range(NUM_PATROL):
    patrol()
    
tello.land()
battery_level(tello)


[INFO] tello.py - 437 - Send command: 'takeoff'


Battery level is 84%


[INFO] tello.py - 461 - Response takeoff: 'ok'
[INFO] tello.py - 437 - Send command: 'up 80'


KeyboardInterrupt: 

In [ ]:
# tello??

### Tentative de vol spirale

* TODO donner de bonnes coordonnées
* TODO fluidité des changement de coordonnées

In [36]:
def move_spirale(t, pos):
    return [np.cos(t)*10, np.sin(t)*10, pos[2]+t, 0]

In [37]:
def spirale(tello):
    TEMPO=0.05
    t=0
    pos=[20, 20, 0, 0]
    speed=50
    PADDING=10

    next_pos = pos
    for k in range(5):
        t+=1
        pos_ = move_spirale(t, next_pos)
        next_pos = difference(next_pos, pos_)
        if next_pos[0] > 0:
            next_pos[0]+=PADDING
        else:
            next_pos[0]-=PADDING
        if next_pos[1] > 0:
            next_pos[1]+=PADDING
        else:
            next_pos[1]-=PADDING
        print(t, next_pos)        
        print('before move')

        tello.go_xyz_speed(int(next_pos[0]), int(next_pos[1]), int(next_pos[2])+PADDING, speed)
        print('move done')
#         time.sleep(TEMPO)


In [39]:
tello = Tello()
tello.connect(False)

# afficher niveau de batterie
battery_level(tello)

tello.takeoff()
# tello.move_up(80)

spirale(tello)
    
tello.land()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'takeoff'


Battery level is 80%


[INFO] tello.py - 461 - Response takeoff: 'ok'
[INFO] tello.py - 437 - Send command: 'go -24 -21 11 50'


1 [-24.5969769413186, -21.585290151921036, 1, 0]
before move


[INFO] tello.py - 461 - Response go -24 -21 11 50: 'ok'
[INFO] tello.py - 437 - Send command: 'go 30 40 12 50'


...

[INFO] tello.py - 461 - Response go 53 51 14 50: 'ok'
[INFO] tello.py - 437 - Send command: 'go -60 -71 15 50'


move done
5 [-60.96237547858325, -71.28828213313128, 5, 0]
before move


[INFO] tello.py - 461 - Response go -60 -71 15 50: 'ok'
[INFO] tello.py - 437 - Send command: 'land'


move done


[INFO] tello.py - 461 - Response land: 'ok'
